In [8]:
from pyarrow.parquet import ParquetFile
import pandas as pd

In [9]:
def read_parquet_data(parquet,cols):
    '''
    will return cols from parquet file in pandas df
    '''
    pq = parquet.read(columns = cols)
    df = pd.DataFrame()
    for c in cols:
        df[c] = pq[c]
        
    return df

In [18]:
# Filepaths
gz_filepath = 'data/gz_decals_auto_posteriors.parquet'

#Open GZ data
parquet = ParquetFile(gz_filepath)
cols = ['iauname','ra','dec']
data = read_parquet_data(parquet,cols)
print(f'There are {len(data)} galaxies in GZ.')

There are 313789 galaxies in GZ.


In [19]:
data.to_csv('gzdecals.csv',index=False)